In [1]:
import sys

def tsp_branch_and_bound(cost_matrix):
    n = len(cost_matrix)
    final_path = [-1] * (n + 1)
    visited = [False] * n
    final_res = sys.maxsize

    def copy_to_final(curr_path):
        nonlocal final_path
        for i in range(n):
            final_path[i] = curr_path[i]
        final_path[n] = curr_path[0]

    def first_min(adj, i):
        min_val = sys.maxsize
        for k in range(n):
            if adj[i][k] < min_val and i != k:
                min_val = adj[i][k]
        return min_val

    def second_min(adj, i):
        first, second = sys.maxsize, sys.maxsize
        for j in range(n):
            if i == j:
                continue
            if adj[i][j] <= first:
                second = first
                first = adj[i][j]
            elif adj[i][j] <= second and adj[i][j] != first:
                second = adj[i][j]
        return second

    def tsp_rec(adj, curr_bound, curr_weight, level, curr_path, visited):
        nonlocal final_res
        print(f"\nLevel: {level}")
        print(f"Current path: {curr_path[:level]}")
        print(f"Visited cities: {visited}")
        print(f"Current bound: {curr_bound}")
        print(f"Current weight: {curr_weight}")

        if level == n:
            if adj[curr_path[level - 1]][curr_path[0]] != 0:
                curr_res = curr_weight + adj[curr_path[level - 1]][curr_path[0]]
                print(f"Completed cycle: {curr_path[:level]} -> {curr_path[0]} with cost {curr_res}")

                if curr_res < final_res:
                    copy_to_final(curr_path)
                    final_res = curr_res
            return

        for i in range(n):
            if adj[curr_path[level - 1]][i] != 0 and not visited[i]:
                temp = curr_bound
                curr_weight += adj[curr_path[level - 1]][i]

                if level == 1:
                    curr_bound -= (first_min(adj, curr_path[level - 1]) + first_min(adj, i)) / 2
                else:
                    curr_bound -= (second_min(adj, curr_path[level - 1]) + first_min(adj, i)) / 2

                # If the current bound + weight is better than the final result
                if curr_bound + curr_weight < final_res:
                    curr_path[level] = i
                    visited[i] = True

                    # Recursively visit the next level
                    tsp_rec(adj, curr_bound, curr_weight, level + 1, curr_path, visited)

                # Backtracking
                curr_weight -= adj[curr_path[level - 1]][i]
                curr_bound = temp
                visited[i] = False

    def tsp(adj):
        curr_bound = 0
        curr_path = [-1] * (n + 1)

        # Initial bound calculation
        for i in range(n):
            curr_bound += (first_min(adj, i) + second_min(adj, i))
        curr_bound = curr_bound // 2

        # Start at the first city
        visited[0] = True
        curr_path[0] = 0

        # Start the recursive TSP
        tsp_rec(adj, curr_bound, 0, 1, curr_path, visited)

        # Final results
        print("\nFinal minimum cost:", final_res)
        print("Path taken:", final_path)

    tsp(cost_matrix)

# Example usage
cost_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]
tsp_branch_and_bound(cost_matrix)


Level: 1
Current path: [0]
Visited cities: [True, False, False, False]
Current bound: 75
Current weight: 0

Level: 2
Current path: [0, 1]
Visited cities: [True, True, False, False]
Current bound: 65.0
Current weight: 10

Level: 3
Current path: [0, 1, 2]
Visited cities: [True, True, True, False]
Current bound: 45.0
Current weight: 45

Level: 4
Current path: [0, 1, 2, 3]
Visited cities: [True, True, True, True]
Current bound: 20.0
Current weight: 75
Completed cycle: [0, 1, 2, 3] -> 0 with cost 95

Level: 3
Current path: [0, 1, 3]
Visited cities: [True, True, False, True]
Current bound: 42.5
Current weight: 35

Level: 4
Current path: [0, 1, 3, 2]
Visited cities: [True, True, True, True]
Current bound: 22.5
Current weight: 65
Completed cycle: [0, 1, 3, 2] -> 0 with cost 80

Level: 2
Current path: [0, 2]
Visited cities: [True, False, True, False]
Current bound: 62.5
Current weight: 15

Final minimum cost: 80
Path taken: [0, 1, 3, 2, 0]
